# 도로명 주소 구하기

In [1]:
import pandas as pd
import numpy as np

In [2]:
guro = pd.read_csv('data/구로구 전처리 데이터.csv')
print(guro.shape)
guro.head()

(291500, 32)


,SN,PROCESS_DT,DLVR_REQUST_ENTRPS_ID,DLVR_REQUST_STTUS_VALUE,DLVR_RCEPT_ENTRPS_ID,DLVR_STORE_ID,DLVR_STORE_INDUTY_NM,DLVR_STORE_LEGALDONG_CODE,DLVR_STORE_BRTC_NM,DLVR_STORE_SIGNGU_NM,...,DLVR_AMOUNT,CALL_RLAY_FEE_AMOUNT,GOODS_AMOUNT,SETLE_KND_VALUE,SETLE_CARD_NM,DLVR_RCEPT_TIME,DLVR_CARALC_TIME,DLVR_RECPT_TIME,DLVR_COMPT_TIME,DLVR_CANCL_TIME
0,3486946,2019-12-01,C000049,1,C000049,S0022458,패스트푸드,1153010800,서울특별시,구로구,...,3000,400,11500,2,NaN,2019-12-01 20:22:17,2019-12-01 20:37:16,2019-12-01 20:44:49,2019-12-01 21:04:42,\N
1,3486960,2019-12-01,C000049,1,C000049,S0022440,패스트푸드,1153010600,서울특별시,구로구,...,3000,400,13600,2,NaN,2019-12-01 22:38:54,2019-12-01 22:40:24,2019-12-01 22:49:55,2019-12-01 22:55:59,\N
2,3486964,2019-12-01,C000049,1,C000049,S0000200,분식,1153010600,서울특별시,구로구,...,3300,300,15000,2,NaN,2019-12-01 18:52:02,2019-12-01 19:12:08,2019-12-01 19:22:07,2019-12-01 19:43:26,\N
3,3486978,2019-12-01,C000049,1,C000049,S0000267,회,1153010600,서울특별시,구로구,...,3300,300,26000,1,승인:삼성카드사,2019-12-01 19:06:12,2019-12-01 19:52:00,2019-12-01 19:57:03,2019-12-01 20:05:32,\N
4,3486982,2019-12-01,C000049,1,C000237,S0000135,치킨,1153010700,서울특별시,구로구,...,4300,400,17900,2,NaN,2019-12-01 23:11:38,2019-12-01 23:13:18,2019-12-01 23:25:19,2019-12-01 23:33:09,\N


In [3]:
# 도로명 주소 데이터 불러오기
geo = pd.read_csv('data/도로명정보조회.csv')
geo.head()

,시군구,도로명번호,도로명,로마자표기,읍면동일련번호,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,시군구 코드,도로명코드,도로명 주소
0,구로구,1000115,수원문산고속도로,Suwon Munsan Expressway,0,서울특별시,구로구,2,NaN,NaN,11530,115301000115,서울특별시 구로구 수원문산고속도로
1,구로구,2000003,남부순환로,Nambusunhwan-ro,0,서울특별시,구로구,2,NaN,NaN,11530,115302000003,서울특별시 구로구 남부순환로
2,구로구,2000003,남부순환로,Nambusunhwan-ro,1,서울특별시,구로구,1,102.0,구로동,11530,115302000003,서울특별시 구로구 남부순환로
3,구로구,2000003,남부순환로,Nambusunhwan-ro,2,서울특별시,구로구,1,103.0,가리봉동,11530,115302000003,서울특별시 구로구 남부순환로
4,구로구,2000003,남부순환로,Nambusunhwan-ro,3,서울특별시,구로구,1,107.0,개봉동,11530,115302000003,서울특별시 구로구 남부순환로


In [4]:
# 가게와 배달 목적지의 도로명 주소 코드를 str 타입으로 변경해주기
guro['DLVR_STORE_RDNMADR_CODE'] = guro['DLVR_STORE_RDNMADR_CODE'].astype('string')
guro['DLVR_DSTN_RDNMADR_CODE'] = guro['DLVR_DSTN_RDNMADR_CODE'].astype('string')
guro['DLVR_DSTN_RDNMADR_CODE'] = guro['DLVR_DSTN_RDNMADR_CODE'].str.split('.').str[0]
geo['도로명코드'] = geo['도로명코드'].astype('string')

In [5]:
# 도로명과 도로명 코드만 필요하기 때문에 indexing 해줌.
geo = geo[['도로명 주소', '도로명코드']]

# 도로명과 도로명 코드가 중복되는 부분을 삭제
geo = geo.drop_duplicates().reset_index(drop = True)
geo.head()

,도로명 주소,도로명코드
0,서울특별시 구로구 수원문산고속도로,115301000115
1,서울특별시 구로구 남부순환로,115302000003
2,서울특별시 구로구 시흥대로,115302005010
3,서울특별시 구로구 안양천로,115303000010
4,서울특별시 구로구 광남로,115303000016


In [6]:
# 가게의 도로명 코드를 기준으로 merge 해줌
temp = pd.merge(guro, geo, left_on = 'DLVR_STORE_RDNMADR_CODE', right_on = '도로명코드', how = 'left')

# 컬럼명 변경
temp.rename(columns = {'도로명 주소' : 'DLVR_STORE_ROAD'}, inplace = True)
temp.rename(columns = {'도로명코드' : 'DLVR_STORE_ROAD_CODE'}, inplace = True)

In [7]:
# 배달 목적지의 도로명 코드를 기준으로 merge 해줌
temp2= pd.merge(temp, geo, left_on = 'DLVR_DSTN_RDNMADR_CODE', right_on = '도로명코드', how = 'left')

# 컬럼명 변경
temp2.rename(columns = {'도로명 주소' : 'DLVR_DSTN_ROAD'}, inplace = True)
temp2.rename(columns = {'도로명코드' : 'DLVR_DSTN_ROAD_CODE'}, inplace = True)

In [8]:
# 도로명 주소가 없는 데이터의 수
temp2[['DLVR_STORE_ROAD', 'DLVR_DSTN_ROAD']].isnull().sum()

DLVR_STORE_ROAD    48998
DLVR_DSTN_ROAD     49001
dtype: int64

In [9]:
temp2.head()
print(temp2.shape)

(291500, 36)
